In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Graph Convolutional Network

https://docs.dgl.ai/en/latest/tutorials/models/1_gnn/1_gcn.html

In [7]:
!pip install -U ipykernel

     |████████████████████████████████| 124 kB 13.8 MB/s 
     |████████████████████████████████| 1.8 MB 87.9 MB/s 
     |████████████████████████████████| 787 kB 28.9 MB/s 
  Attempting uninstall: ipython
    Found existing installation: ipython 7.19.0
    Uninstalling ipython-7.19.0:
      Successfully uninstalled ipython-7.19.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.3.4
    Uninstalling ipykernel-5.3.4:
      Successfully uninstalled ipykernel-5.3.4
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [2]:
!conda install -y -c dglteam dgl 

Solving environment: done

## Package Plan ##

  environment location: /home/jongoon/anaconda3

  added / updated specs:
    - dgl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dgl-0.5.3                  |           py38_0         3.4 MB  dglteam
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

  dgl                dglteam/linux-64::dgl-0.5.3-py38_0



dgl-0.5.3            | 3.4 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
conda install -y pytorch torchvision torchaudio cpuonly -c pytorch

Solving environment: done

## Package Plan ##

  environment location: /home/jongoon/anaconda3

  added / updated specs:
    - cpuonly
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cpuonly-1.0                |                0           2 KB  pytorch
    libuv-1.40.0               |       h7b6447c_0         736 KB
    ninja-1.10.2               |   py38hff7bd54_0         1.4 MB
    pytorch-1.7.1              |      py3.8_cpu_0        59.5 MB  pytorch
    torchaudio-0.7.2           |             py38         9.8 MB  pytorch
    torchvision-0.8.2          |         py38_cpu        17.2 MB  pytorch
    ------------------------------------------------------------
                                           Total:        88.7 MB

The following NEW packages will be INSTALLED:

  cpuonly            pytorch/noarch::cpuonly-1.0-0
  libuv        

In [1]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

Using backend: pytorch


In [2]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [4]:
from dgl.data import CoraGraphDataset
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask

In [5]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Epoch 00000 | Loss 1.9574 | Test Acc 0.2220 | Time(s) nan
Epoch 00001 | Loss 1.8155 | Test Acc 0.2840 | Time(s) nan
Epoch 00002 | Loss 1.6953 | Test Acc 0.3350 | Time(s) nan


/home/jongoon/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jongoon/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00003 | Loss 1.5821 | Test Acc 0.4790 | Time(s) 0.0144
Epoch 00004 | Loss 1.4799 | Test Acc 0.5610 | Time(s) 0.0139
Epoch 00005 | Loss 1.3834 | Test Acc 0.6200 | Time(s) 0.0141
Epoch 00006 | Loss 1.2904 | Test Acc 0.6740 | Time(s) 0.0138
Epoch 00007 | Loss 1.1989 | Test Acc 0.7130 | Time(s) 0.0149
Epoch 00008 | Loss 1.1047 | Test Acc 0.7440 | Time(s) 0.0146
Epoch 00009 | Loss 1.0160 | Test Acc 0.7460 | Time(s) 0.0150
Epoch 00010 | Loss 0.9338 | Test Acc 0.7500 | Time(s) 0.0146
Epoch 00011 | Loss 0.8558 | Test Acc 0.7470 | Time(s) 0.0143
Epoch 00012 | Loss 0.7808 | Test Acc 0.7360 | Time(s) 0.0142
Epoch 00013 | Loss 0.7108 | Test Acc 0.7370 | Time(s) 0.0143
Epoch 00014 | Loss 0.6473 | Test Acc 0.7370 | Time(s) 0.0147
Epoch 00015 | Loss 0.5875 | Test Acc 0.7450 | Time(s) 0.0145
Epoch 00016 | Loss 0.5324 | Test Acc 0.7460 | Time(s) 0.0148
Epoch 00017 | Loss 0.4831 | Test Acc 0.7470 | Time(s) 0.0150
Epoch 00018 | Loss 0.4378 | Test Acc 0.7510 | Time(s) 0.0149
Epoch 00019 | Loss 0.395